In [1]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [2]:
model = ChatOpenAI(model='gpt-5-mini')
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})

In [3]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [4]:
loader = PyPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
docs = loader.load()
full_text = format_docs(docs)
text_splitter = SemanticChunker(embeddings=embeddings, breakpoint_threshold_type='percentile', breakpoint_threshold_amount=70)
docs = text_splitter.create_documents([full_text])
for doc in docs:
    doc.metadata['source'] = '2024_KB_부동산_보고서_최종.pdf'

print(len(docs))

286


In [5]:
persist_dir = './data/rag_practice_chroma_2'
if not os.path.exists(persist_dir):
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_dir
    )
else:
    vectorstore = Chroma(
        embedding_function=embeddings,
        persist_directory=persist_dir
    )
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

print(vectorstore._collection.count())

286


In [6]:
template = '''
당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
정보에 없는 내용은 "정보가 없어 답변할 수 없습니다." 라고 답변해주세요.
컨텍스트: {context}
'''

prompt = ChatPromptTemplate.from_messages(
    [('system', template),
     ('placeholder', '{chat_history}'),
     ('human', '{question}')]
)

In [7]:
chain = (
    RunnablePassthrough.assign(context=lambda x: format_docs(retriever.invoke(x['question'])))
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [9]:
def chat_with_bot():
    session_id = 'user_session'
    print('KB 부동산 보고서 챗봇입니다. 질문해주세요. (종료하려면 "종료"를 입력해주세요.)')

    while True:
        user_input = input('사용자: ')
        if user_input == '종료':
            break

        response = chain_with_memory.invoke(
            {'question':user_input},
            {'configurable':{'session_id':session_id}}
        )

        print('챗봇: ', response)

In [10]:
chat_with_bot()

KB 부동산 보고서 챗봇입니다. 질문해주세요. (종료하려면 "종료"를 입력해주세요.)
챗봇:  질문하신 “올해(2024년) 부동산 매매 전망”은 KB 리포트 설문 결과를 기준으로 요약하면 다음과 같습니다.

요약 전망
- 전반적 전망: 하락세 지속 전망이 우세. 부동산시장 전문가의 74%, 공인중개사와 PB 각 79%가 2024년 전국 주택 매매가격이 하락할 것으로 응답.  
- 하락폭(설문 응답 기준): 전문가·공인중개사 다수는 1~3% 하락을, PB는 상대적으로 큰 폭인 3~5% 하락 전망 비율이 높음.  
- 다만 급락 우려는 다소 완화되어 ‘상승 전망’ 비율은 2023년보다 증가(전문가 +21%p, 공인중개사 +17%p, PB +13%p).

주요 변수(향후 흐름에 영향)
- 금리: 금리 인하 기대가 매수 심리 회복 요인이나, 인하 시기·폭이 관건.  
- 정책: 규제 완화(특히 재건축·재개발 관련)와 실수요자 금융지원 여부에 따라 회복 강도 결정.  
- 공급·거래: 2023년 거래량은 역대 최저권 수준으로, 공급(분양·인허가)도 급감 중이라 지역별 수급 불균형 가능성.  
- 법·사업 리스크: 「노후계획도시 특별법」 등으로 재건축·재개발 대상 확대 시 시장 영향 확대 가능.  
- 전세·입주: 아파트 전세 수요 집중과 수도권 입주물량 감소로 전세가격 상승 압력 존재.  
- 가계부채·주담대: 주택담보대출은 늘고 있으나 가계부채 관리 강화가 정책 추진의 제약 요인.

결론
- 설문 결과 기준으로는 “2024년에도 전반적 하락 우세, 다만 폭은 제한적(주요 응답자 기준 1~5% 수준)”이라는 전망이 지배적입니다. 구체적 지역·가격대별 전망이나 최신 금리·정책 변화에 따른 즉각적 예측은 리포트에 추가 정보가 없어 답변할 수 없습니다.
